In [39]:
import pandas as pd

data = pd.read_csv('data/train.csv')

data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [40]:
single_row = data.iloc[28:29]  # Возвращает DataFrame с одной строкой

In [41]:
single_row = single_row.drop('price_doc', axis=1)

In [ ]:
single_row['product_type']

single_row = single_row.fillna('Na')


In [32]:
import json
for_json = single_row.to_dict('records')
json_string = json.dumps(for_json)

In [33]:
with open('output.json', 'w') as f:
    json.dump(for_json, f, indent=4)

In [85]:
single_row.shape

(1, 292)

In [86]:
data.shape

(30471, 292)

In [42]:
import json
from sklearn.preprocessing import LabelEncoder

def preprocess_data_Occupier(input_data: pd.DataFrame) -> pd.DataFrame:
        
        with open('removed_columns.txt', 'r') as f:
            removed_columns = [line.strip() for line in f.readlines()]

        with open('mean_values.json', 'r') as f:
            mean_values = json.load(f)

        data = pd.DataFrame(input_data)

        data = data.drop([
            'ID_metro',
            'ID_railroad_station_walk',
            'ID_railroad_station_avto',
            'ID_big_road1',
            'ID_big_road2',
            'ID_railroad_terminal',
            'ID_bus_terminal'
        ], axis=1)

        data = data.drop(columns=removed_columns, errors='ignore')

        data = data.drop('id', axis=1)

        numeric_columns = data.loc[:, data.dtypes!='object'].columns

        for col in numeric_columns:
            if col in mean_values:
                data[col] = data[col].fillna(mean_values[col])

        categorical_columns = data.loc[:, data.dtypes == 'object'].columns

        label_encoder = LabelEncoder()

        # Применяем Label Encoding для категориальных столбцов
        for col in categorical_columns:
            if col != 'timestamp':
                # Применяем LabelEncoder
                data[col] = label_encoder.fit_transform(data[col])
        
        data['timestamp'] = pd.to_datetime(data['timestamp'])

        data['month'] = data.timestamp.dt.month
        data['year'] = data.timestamp.dt.year

        data = data.sort_values(['timestamp'])
        
        data['year'] = label_encoder.fit_transform(data['year'])

        data['year'] = label_encoder.fit_transform(data['month'])

        data = data.drop('timestamp', axis=1)

        Occupier = data[data['product_type'] == 0].copy()

        return Occupier


In [43]:
single_row = preprocess_data_Occupier(single_row)

C:\Users\denis\AppData\Local\Temp\ipykernel_20904\3010955360.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['month'] = data.timestamp.dt.month
C:\Users\denis\AppData\Local\Temp\ipykernel_20904\3010955360.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['year'] = data.timestamp.dt.year


In [44]:
single_row.shape

(1, 150)

In [90]:
single_row['full_sq'].dtype

dtype('int64')

In [45]:
import numpy as np



def load_model(path: str):
        import joblib
        return joblib.load(path)

test_model = load_model('xg_model_Occupier.pkl')

print("Предсказание: ", np.exp(test_model.predict(single_row.head(1))))

Предсказание:  [8526827.]
